In [18]:
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait # 로딩까지 기다리는
from selenium.webdriver.support import expected_conditions as EC # #예외상황이 떴을 때 대처하는 라이브러리
from selenium.webdriver.chrome.options import Options #필요한 옵션을 가져옴

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys #키보드
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import tqdm
import pyperclip
import time
import csv
import random

In [19]:
# 크롤링 시작과 끝 날짜(월별로 크롤링)
first_days = pd.date_range('2020/01/01', '2022/10/31', freq="MS") #각 월의 첫 날
last_days = pd.date_range('2020/01/01', '2022/10/31', freq="M") #각 월의 마지막 날

keyword_list = ["펫택시 불편", "펫택시 별로"]

In [20]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage') #overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드


chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치


In [21]:
# 블로그 url 수집
blog_url_list = []

# 네이버에 키워드 검색
for keyword in  keyword_list:
    url = f'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={keyword}&oquery={keyword}%27'
    
    driver = webdriver.Chrome(executable_path=r"C:\Developes\python\고객분석\chromedriver", options=chrome_options)
    driver.implicitly_wait(2)
    
    driver.get(url)
    time.sleep(1)
    
    url_want = 990
    
    # 옵션 버튼
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a').click()
    
    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()            
       
            # 스크롤 조작
            last_heigt = driver.execute_script('return document.body.scrollHeight') # 몇 번을 스크롤해야 끝까지 내려가는지 횟수 반환
            scroll = (last_heigt/30)-1 # 한 페이지에 기본 30개의 글이 있음
            
            # 스크롤 다운
            for a in range(int(scroll)):
                # 끝까지 스크롤 다운
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(1, 1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight;')
                
                if new_height == last_heigt:
                    break
                last_heigt = new_height
            
            soup = bs(driver.page_source)
            blog_url = soup.select('a.api_txt_lines.total_tit')
            
            # url주소 수집
            for b in blog_url:
                blog_url_list.append(b['href']) # 주소 수집
                # print(f'{first_days.year[k]} 키워드 {first_days.month[k]}') 
                driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass 
        
    driver.quit()
                

df = pd.DataFrame({"blog_url": blog_url_list})
df.to_excel("크롤링 url 저장.xlsx", encoding='utf-8-sig', index=False)

In [22]:
len(blog_url_list)
# df.head()

732

In [23]:
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, WebDriverException
import re
from tqdm import tqdm

no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in tqdm(blog_url_list[:50]): #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

 20%|██        | 10/50 [00:59<03:52,  5.82s/it]

10개 블로그 크롤링 완료


 40%|████      | 20/50 [01:59<02:47,  5.59s/it]

20개 블로그 크롤링 완료


 60%|██████    | 30/50 [02:58<01:51,  5.58s/it]

30개 블로그 크롤링 완료


 80%|████████  | 40/50 [04:00<01:01,  6.14s/it]

40개 블로그 크롤링 완료


100%|██████████| 50/50 [04:57<00:00,  5.96s/it]

50개 블로그 크롤링 완료


In [24]:
len(time_list)

50

In [25]:
data = {'time' : time_list,
        'review' : review_list,
        'comment' : comment_list,
        'like' : like_list,
        'url' : url_list
        }

df = pd.DataFrame(data)

print('Duration: ', time.time() - start)
print('Total {} of blog crawling are completed'.format(count))

Duration:  298.8647401332855
Total 50 of blog crawling are completed


In [26]:
df

,time,review,comment,like,url
0,2020. 1. 10. 21:21,개머니즘 아메리칸불리 도트 펫택시 타고 구청에 동물등록하러 갑...,[],[],https://blog.naver.com/pet5120/221767146783
1,2020. 1. 19. 8:14,리 뷰 고양이 병원가려고 반려동물 펫택시 펫미업 예약했어요 ...,[],20,https://blog.naver.com/cmmmco12/221777312259
2,2020. 1. 14. 20:33,터키쉬앙고라 대봉이 [가족이필요해요💕] 대봉이 전발치 후 소...,[],[],https://blog.naver.com/coolyumi056/221771738320
3,2020. 1. 20. 8:30,관련정보 강아지 보조기전문 네발로에서 제 27회 코리아펫쇼!!...,[],13,https://blog.naver.com/nebalroo/221777469042
4,2020. 1. 15. 11:57,"공지사항 실내운동장/놀이방 운영중지~~(광명, 금천, 구로 애...",[],2,https://blog.naver.com/checkindog/221772476810
5,2020. 1. 12. 20:58,댕고카 서비스 댕고카가 왜 편안하고 쾌적한지 설명드립니다.^^...,[],0,https://blog.naver.com/dangocar/221769192135
6,2020. 1. 31. 11:55,반려견 생활 팁 펫시터 or 애견호텔 or 방문펫시터 or 동...,[],5,https://blog.naver.com/rainbow-garden/22179115...
7,2020. 1. 23. 11:03,"일상 대구남구 애견호텔,놀이방,미용 ""반려견들의 1등 휴식처""...",[],6,https://blog.naver.com/cosoulso/221782664410
8,2020. 2. 25. 15:36,관련상식 서울 펫택시 이동픽업서비스 모아봤어요 ...,[],27,https://blog.naver.com/kmmama2019/221824154540
9,2020. 2. 22. 3:08,용용이 일상로그 헬로우펫 [ 펫택시 / 애견택시 들어보셨나요 ...,[],42,https://blog.naver.com/shdmdgk/221819157979


In [27]:
df.to_excel("200101_221031_펫택시.xlsx", index=True, encoding='utf-8-sig')